In [70]:
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import os

In [104]:
xgboost_model = joblib.load('xgboost.pkl')

test_accuracy_data = pd.read_csv('test_target.csv')
test_data = pd.read_csv('test_data_undersampled.csv').drop(columns=['Unnamed: 0'])

test_data['Service Type'] = test_data['Service Type'].map({'Van' : 1, 'Motorcycle' : 0})
test_data['Response Distance'] = test_data['Response Distance'].round().fillna(0.0).astype(int)
test_data['Response Distance'] = test_data['Response Distance'] / 100

In [ ]:
xgboost_model.predict([-0.90986512, -0.75978697,  0.29754476,  1.71774662,  1.00731086,
       -0.35633625, -0.05579893])

In [160]:
def predict_with_xgboost(df):
    
    sc = StandardScaler().fit_transform(df)
    
    xgboost_model = joblib.load('xgboost.pkl')
    
    xgboost_pred = xgboost_model.predict(sc)
    
    predictions = pd.DataFrame(data=xgboost_pred, columns=['Outcome'])
    
    accuracy = accuracy_score(test_accuracy_data.values, predictions)
    
    report = classification_report(test_accuracy_data.values, predictions)
    
    probability = xgboost_model.predict_proba(sc)
    
    #predictions['Outcome'] = predictions['Outcome'].map({'0' : 'Fulfilled', '1' : 'Cancelled'})
    
    return round(accuracy *100, 2) , report, probability

In [161]:
# LETS TEST THE MODEL WITH TEST DATA

print(test_data.head())

   Available Drivers  Response Distance  Service Type  day  tempC  \
0                 13              45.30             1    2     31   
1                210              29.08             1    2     31   
2                 50              23.59             0    5     32   
3                143              35.02             1    1     31   
4                312              17.68             1    2     31   

   windspeedKmph  cloudcover  
0              5          13  
1              5          13  
2             13          17  
3             11          24  
4              5          13  


In [168]:
#passing the test data here
accuracy, report, completion_probability = predict_with_xgboost(test_data)

print('Accuracy score ', accuracy)
print('\n')
print('Classification report \n\n', report)
print('\n')
print('Completion probability \n', completion_probability)


Accuracy score  67.68


Classification report 

               precision    recall  f1-score   support

           0       0.62      0.92      0.74       783
           1       0.83      0.43      0.57       767

    accuracy                           0.68      1550
   macro avg       0.73      0.67      0.66      1550
weighted avg       0.73      0.68      0.66      1550



Completion probability 
 [[0.7130069  0.2869931 ]
 [0.6885272  0.31147274]
 [0.53976977 0.46023026]
 ...
 [0.05363178 0.9463682 ]
 [0.1628412  0.8371588 ]
 [0.62575376 0.37424624]]
